In [ ]:
import great_expectations as ge
from great_expectations.data_context import FileDataContext
from great_expectations.checkpoint import CheckpointConfig
import os
import shutil

# Step 1: Clean slate (optional) – delete existing GE folder if needed
if os.path.exists("great_expectations"):
    shutil.rmtree("great_expectations")

# Step 2: Initialize Great Expectations
context = ge.get_context()

# Step 3: Define your data path
data_path = "data/my_dataset.csv"
assert os.path.exists(data_path), f"{data_path} not found."

# Step 4: Create expectation suite
suite_name = "basic_validation"
context.add_or_update_expectation_suite(expectation_suite_name=suite_name)

# Step 5: Create a batch request from a file
batch_request = {
    "datasource_name": "default_pandas_datasource",
    "data_connector_name": "default_runtime_data_connector_name",
    "data_asset_name": "my_dataset",
    "runtime_parameters": {"path": data_path},
    "batch_identifiers": {"default_identifier_name": "default_id"},
}

# Step 6: Load data and add expectations
validator = context.get_validator(
    batch_request=batch_request, expectation_suite_name=suite_name
)
columns_to_check = validator.columns  # Check all columns for completeness

for col in columns_to_check:
    validator.expect_column_values_to_not_be_null(col)

# Save expectations
validator.save_expectation_suite(discard_failed_expectations=False)

# Step 7: Set up a checkpoint
checkpoint_name = "basic_validation_checkpoint"
checkpoint_config = CheckpointConfig(
    name=checkpoint_name,
    config_version=1.0,
    class_name="Checkpoint",
    run_name_template="%Y%m%d-%H%M%S",
    validations=[{"batch_request": batch_request, "expectation_suite_name": suite_name}],
)
context.add_or_update_checkpoint(**checkpoint_config.to_dict())

# Step 8: Run the checkpoint
results = context.run_checkpoint(checkpoint_name=checkpoint_name)

# Output result summary
print("Validation Success:", results["success"])